*Ref. Recommender System Handbook (Springer)*

#### Must Reed Chapter
- Neighborhood-based Recommendation 107-140
- *Advances in Collaborative Filtering 145-184* อาจมาดูหลัง June ถ้าเวลาไม่พอ
- *Context-Aware RS 217-250*
- Applications and Evaluation of RSs 257(260-267)-294
- a Real Large-ScaleProductionEnvironment 299-329

# 1 Neighborhood-based Recommendation (114)

### Example Data

In [24]:
import pandas as pd

items = pd.DataFrame(data={'item_id': [1, 2, 3, 4, 5], 'name': ['The Matrix', 'Titanic', 'Die Hard', 'Forrest Gump', 'Wall-E']})
items

,item_id,name
0,1,The Matrix
1,2,Titanic
2,3,Die Hard
3,4,Forrest Gump
4,5,Wall-E


In [25]:
users = pd.DataFrame(data={'user_id': [1, 2, 3, 4], 'name': ['John', 'Lucy', 'Eric', 'Diane']})
users

,name,user_id
0,John,1
1,Lucy,2
2,Eric,3
3,Diane,4


In [26]:
rating = pd.DataFrame(data={'user_id': [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4], 'item_id': [1,2,4,5,1,2,3,4,5,1,3,4,5,1,2,3,4], 'score': [5,1,2,2,1,5,2,5,5,2,3,5,4,4,3,5,3]})
rating.head()

,item_id,score,user_id
0,1,5,1
1,2,1,1
2,4,2,1
3,5,2,1
4,1,1,2


In [32]:
rating_x_users = pd.merge(rating, users, how='outer', on=['user_id'])
rating_x_users_x_items = pd.merge(rating_x_users, items, how='outer', on=['item_id'])
rating_x_users_x_items

,item_id,score,user_id,name_x,name_y
0,1,5,1,John,The Matrix
1,1,1,2,Lucy,The Matrix
2,1,2,3,Eric,The Matrix
3,1,4,4,Diane,The Matrix
4,2,1,1,John,Titanic
5,2,5,2,Lucy,Titanic
6,2,3,4,Diane,Titanic
7,4,2,1,John,Forrest Gump
8,4,5,2,Lucy,Forrest Gump
9,4,5,3,Eric,Forrest Gump


Pivot Table

In [35]:
pivot_table = rating_x_users_x_items.pivot(index='name_x', columns='name_y', values='score')
pivot_table

name_y,Die Hard,Forrest Gump,The Matrix,Titanic,Wall-E
name_x,,,,,
Diane,5.0,3.0,4.0,3.0,NaN
Eric,3.0,5.0,2.0,NaN,4.0
John,NaN,2.0,5.0,1.0,2.0
Lucy,2.0,5.0,1.0,5.0,5.0


## 1.1 Item-based Recommendation (117)

เมื่อ  $N_{u}(i)$ คือ score ที่ user $u$ ให้กับ item $i$

และ predicted score ที่ user $u$ ให้กับ $i$ คำนวณจากค่าเฉลี่ยถ่วงน้ำหนัก(a weighted average)ที่ user $u$ เคยให้ไว้กับ item อื่นๆ (น้ำหนักมากหรือน้อยขึ้นกับความเหมือนของ item)

จะได้

$$\hat{r}_{ui} = \frac{\sum_{j \in N_{u}(i)} w_{ij}r_{uj} }{\sum_{j \in N_{u}(i)} |w_{ij}| }$$

สมมติว่าจากข้อมูลเราสนใจ predict rating ของ Eric ที่มีต่อ Titanic โดยเมื่อพิจารณา**ความเหมือน**ของ item ด้วย neareast-neighbors พบว่า item ที่เหมือน Titanic มากที่สุดคือ Forrest Gump และ Wall-E โดยมีค่า similarity อยู่ที่ 0.85 และ 0.75 ตามลำดับ เมื่อ Eric ให้ Score Forrest Gump ที่ 5 และ Wall-E ที่ 4 

จะได้ $$\hat{r} = \frac{0.85x5 + 0.75x4}{0.85+0.75} \approx 4.53$$

ทีนี้ตอนที่จะทำให้หลายๆคนพร้อมๆกัน ปัญหาอย่างนึงที่จะเจอคือ แต่ละคนมีมาตรฐานแตกต่างกัน บางคนรู้สึกดีให้ score 3 ขณะที่บางคนให้ 5 ดังนั้นเพื่อปรับ scale score ให้เหมือนๆกันก็ต้องทำ normalization กับ rating score ของแต่ละคนก่อนจะได้

$$\hat{r}_{ui} = h^{-1}\Bigg(\frac{\sum_{j \in N_{u}(i)} w_{ij}h(r_{uj}) }{\sum_{j \in N_{u}(i)} |w_{ij}| }\Bigg)$$

สมการข้างบนคือ prediction rating แบบ **Item-based Regression** เมื่อ data score ที่เราเก็บมาเป็น continuous แต่หากเป็น data score เป็น discrete เช่น like, unlike หรือ buy, not buy ก็จะต้อง predict score ด้วย **Item-based Classification**

### Item-based Classification

ในกรณีนี้ เมื่อ j คือ items ที่ user เคยให้ score ไว้แล้ว i คือ item ใหม่ที่ยังไม่มี score และเราต้องการ predict จากน้ำหนักความ similarity ระหว่าง i และ j ได้สมการที่ normalized แล้วของ Item-based Classification ดังนี้

$$\hat{r}_{ui} = h^{-1}\Bigg(  \arg\max_{r \in S'} \sum_{j \in N_{u}(i)} \delta(h(r_{uj}) = r)w_{ij} \Bigg)$$

ซึ่งคือการหา r ที่ทำให้ $\sum_{j \in N_{u}(i)} \delta(h(r_{uj}) = r)w_{ij}$ มีค่ามากที่สุด

เมื่อ $\delta(r_{vi} = r)$ is 1 if $r_{vi} = r$ ,and 0 otherwise.

## 1.2 Component of Neighborhood Methods (120-141)

### 1.2.1 Rating Normalization (121)

#### 1.2.1.1 Mean-centering (121)

$$h(r_{ui}) = r_{ui} - \bar{r}_{i}$$

เมื่อ $\bar{r}_{i}$ คือ ค่าเฉลี่ยของ score ที่ user $u$ ให้กับ item $i$ ต่างๆ

สุดท้ายเมื่อนำไป predict ค่า $\hat{r}_{ui}$ จะได้

$$\hat{r}_{ui} = h^{-1}\Bigg(\frac{\sum_{j \in N_{u}(i)} w_{ij}h(r_{uj}) }{\sum_{j \in N_{u}(i)} |w_{ij}| }\Bigg) = \bar{r}_i + \frac{\sum_{j \in N_{u}(i)} w_{ij}(r_{uj} - \bar{r}_{j}) }{\sum_{j \in N_{u}(i)} |w_{ij}| } $$

สร้าง function หาค่า $h(r_{ui})$

**Before**

In [38]:
pivot_table

name_y,Die Hard,Forrest Gump,The Matrix,Titanic,Wall-E
name_x,,,,,
Diane,5.0,3.0,4.0,3.0,NaN
Eric,3.0,5.0,2.0,NaN,4.0
John,NaN,2.0,5.0,1.0,2.0
Lucy,2.0,5.0,1.0,5.0,5.0


**After**

ดึงมาเฉพาะค่าที่เป็นเลข

In [110]:
item_scores = pivot_table.values
item_scores

array([[  5.,   3.,   4.,   3.,  nan],
       [  3.,   5.,   2.,  nan,   4.],
       [ nan,   2.,   5.,   1.,   2.],
       [  2.,   5.,   1.,   5.,   5.]])

สร้างฟังก์ชั่นที่หา mean ของแต่ละ array (คิดเฉพาะที่มีค่า ไม่คิด nan)

In [128]:
import numpy as np

def mean_of_array(x):
    valid_value = x[~np.isnan(x)]
    return np.mean(valid_value)

user mean-centered ratings

In [129]:
mean_of_each_row = np.apply_along_axis(mean_of_array , axis=1, arr=item_scores)
print(mean_of_each_row)
user_mean_centered_rating = (item_scores.transpose() - mean_of_each_row).transpose()
user_mean_centered_rating

[ 3.75  3.5   2.5   3.6 ]


array([[ 1.25, -0.75,  0.25, -0.75,   nan],
       [-0.5 ,  1.5 , -1.5 ,   nan,  0.5 ],
       [  nan, -0.5 ,  2.5 , -1.5 , -0.5 ],
       [-1.6 ,  1.4 , -2.6 ,  1.4 ,  1.4 ]])

ผลถูกต้องตามเฉลย

item mean-centered ratings

In [132]:
mean_of_each_column = np.apply_along_axis(mean_of_array , axis=0, arr=item_scores)
print(mean_of_each_column)
item_mean_centered_rating = item_scores - mean_of_each_column
item_mean_centered_rating

[ 3.33333333  3.75        3.          3.          3.66666667]


array([[ 1.66666667, -0.75      ,  1.        ,  0.        ,         nan],
       [-0.33333333,  1.25      , -1.        ,         nan,  0.33333333],
       [        nan, -1.75      ,  2.        , -2.        , -1.66666667],
       [-1.33333333,  1.25      , -2.        ,  2.        ,  1.33333333]])

ผลถูกต้องตามเฉลย

#### 1.2.1.2 Z-score normalization (122)

### 1.2.2 Similarity Weight Computation

### 1.2.3 Neighborhood Selection

### 1.2.4 Advanced Techniques

- Bayesian Clustering 
- Latent Semantic Analysis
- Latent Dirichlet Allocation
- Maximum Entropy
- Boltzmann Machines
- SVMs
- Singular Value Decomposition 